In [4]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import gc
import math

##  function Extract_NER():   Extract NER from receiving text list

In [5]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [11]:
#讀檔

df_file = pd.read_csv(r"/Users/wombat/Desktop/課程筆記/政治與資訊/NSYSU1082（莊老師）/Lab/KCC Data/Nwes/News500.csv")
print(df_file.columns)
print("-----------------------------------------------------------")
df_file.head(3)
df_file.columns = ['index','file','Text']
df_file = df_file.iloc[0:20,]

Index(['1', '2016/4/18',
       '馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳宏謀等人也一同到場，聽取衛武營興建工程簡報、瞭解工程進度與執行情形，迄今已完成逾百分之九十五的工程進度，共計投入新臺幣一百零五億八千萬元經費，預計今年六月竣工、以明年開館營運為目標。馬英九總統指出，衛武營這個案子是從九十四年時編列預算核定施工，到正式完工的時間長達十三年，從原先預計的五年，一差就差了八年，其中就和故宮南院遇到相同問題，對於一個沒有工程單位的的部門要如何克服這種困難，值得大家共同檢討。馬總統致詞時表示，民國九十九年親自主持動土典禮，看著原從一片黃土，到如今生長出有如魟魚般悠遊於海浪的建築，實在令人驚豔，從無到有的過程中，包含多少團隊的付出及努力，雖然籌建過程非常艱辛，但在各界不斷的關注及投入下，今天看到新建工程已幾近完工，藉此特別肯定文化部在籌建過程中付出的辛勞，高雄市政府及各界的大力支持配合。總統並強調，只有文化才能讓一個城市變偉大，期許衛武營藝術文化中心未來不僅是臺灣藝術力量的南方集結地，也將與臺北國家兩廳院、臺中國家歌劇院攜手並進，共同打造台灣在國際藝文版圖的新地標。文化部長洪孟啟則表示，未來衛武營最大的考驗是需面臨營運壓力，因此文化部將以衛武營作為南部的文化中心，往上結合嘉義文化園區與故宮南院，以及台南地方藝文團體，往下則與高雄大東文化藝術中心、屏東演藝廳結合成文化廊帶，未來衛武營更將與國際接軌，文化部已邀請台灣旅德指揮家簡文彬擔任衛武國家藝術文化中心藝術總監，並積極接洽國外藝文團隊，希望國際型表演節目未來第一站就到高雄演出，讓衛武營成為台灣重要的文化指標場館。'],
      dtype='object')
-----------------------------------------------------------


#df_file = df_file.iloc[0:50,]
#只跑0-50

## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [13]:
#逐筆傳遞欄位 "text"
df_text = df_file["Text"]
print("Text 欄位筆數 = ",len(df_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(df_text)):
# for i in range(6):
    rowCunt += 1
    ls_dfRow = [df_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(df_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        # NER_df.to_csv(r"News Data\News NER Out\\NewsNER870_" + str(fileCunt)+ ".csv")
        NER_df.to_csv(r"ner out/NerTest_" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


Text 欄位筆數 =  20
------------------------------------------
Row No =  0
Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10
Row No =  11
Row No =  12
Row No =  13
Row No =  14
Row No =  15
Row No =  16
Row No =  17
Row No =  18
Row No =  19
fileCunt No =  2
Totle NER Files =  2


## 3. Integrate all csv files into single data frame

In [14]:
import glob
# import pandas as pd

# path = r'News Data\News NER OUT'
path = r'ner out'
allFiles = glob.glob("{}/*.csv".format(path))
li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby('Class').count()

368


,NER
Class,
EVENT,7
GPE,58
LAW,3
LOC,9
NORP,16
ORG,153
PERSON,122


## 4. Drop duplicates, group by class and write into csv file 

In [15]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('Class').count()

259


,NER
Class,
EVENT,6
GPE,34
LAW,2
LOC,9
NORP,10
ORG,100
PERSON,98


In [16]:
# ddFrame.sort_values(by=['Class']).to_csv(r"News Data\News NER OUT\\NewsNER.csv")
ddFrame.sort_values(by=['Class']).to_csv(r"ner out/NerTest.csv")